In [1]:
import random
import math
import operator
import pandas as pd
import numpy as np
from numba import njit

In [2]:
#字典类型，保存user对item的tag,即{userid:{item1:[tag1,tag2],...}}
records =dict()
#训练集，测试集
train_data = dict()
test_data = dict()
#用户标签，商品标签
user_tags = dict()
# 标签-商品
tag_items = dict()
#用户-物品
user_items = dict()
#标签-用户
tag_users = dict()

In [3]:
#数据加载
def loadData():    
    print("开始数据加载...")
    file_path = "./user_taggedbookmarks-timestamps.dat"
    df = pd.read_csv(file_path,sep="\t")
    #print(df)
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]
        #键不存在时，设置默认值{}
        records.setdefault(uid,dict())
        records[uid].setdefault(iid,list())        
        records[uid][iid].append(tag)
    print('数据集大小为%d'%(len(df)))
    print('设置tag的人数%d'%(len(records)))
    print("数据加载完成\n")

In [4]:
#将数据集拆分为训练集和测试集
def trainTestSplit(ratio,seed = 100):
    random.seed(seed)
    for u in records.keys():
        for i in records[u].keys():
            #ratio比例设置为测试集
            if random.random() < ratio:
                test_data.setdefault(u,{})
                test_data[u].setdefault(i,[])
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)
    print('训练集样本数 %d,测试集样本数 %d'%(len(train_data),len(test_data)))

In [5]:
#设置矩阵mat[index,item] = 1
def addValueToMat(mat,index,item,value=1):
    if index not in mat:
        mat.setdefault(index,{})
        mat[index].setdefault(item,value)
    else:
        if item not in mat[index]:
            mat[index][item] = value
        else:
            mat[index][item] += value          


In [6]:
#使用训练集，初始化user_tags,tag_items,user_items,tag_users
def initStat():
    #使用训练集
    for u,items in train_data.items():
        for i,tags in items.items():
            for tag in tags:
                #用户和tag的关系
                addValueToMat(user_tags,u,tag,1)
                #tag和item关系
                addValueToMat(tag_items,tag,i,1)
                #用户和item的关系
                addValueToMat(user_items,u,i,1)  
                #标签和用户的关系
                addValueToMat(tag_users,tag,u,1)
    print("user_tags,tag_items,user_items,tag_users初始化完成。")
    print("user_tags大小 %d,tag_items大小 %d,user_items大小 %d,tag_users大小 %d" \
          %(len(user_tags),len(tag_items),len(user_items),len(tag_users)))

In [7]:
#对用户user推荐topN
#@njit(fastmath = True)
def recommend(method,user,N):
    recommend_items = dict()
    tagged_items = user_items[user]
    #wut:用户对某标签的使用次数；wti:商品被打上相同标签的次数
    
    if method == "NormTagBased":
        #NormTagBased分数为所有的(wut/user_tag_csum)*(wti/tag_item_csums)之和
        #(user_tag_csum:该用户打的所有标签次数，tag_item_csums:物品打上该标签总的次数)         
        user_tag_csum = 0        
        #计算用户累计标签次数
        for tag,wut in user_tags[user].items():
            user_tag_csum += wut
            
        tag_item_csums = dict()         
        #统计标签累计使用次数
        for tag,items in tag_items.items():
            for item,wti in items.items():
                if tag not in tag_item_csums.keys():
                    tag_item_csums[tag] = wti
                else:
                    tag_item_csums[tag] +=wti
                    
    if method == "TagBased_TFIDF":
        #TagBased_TFIDF分数为所有的(wut/log(1 + tag_user_csums))*(wti)之和（tag_user_csums:所有使用该标签的用户）
        tag_user_csums = dict()
        #统计标签不同用户数
        for tag,users in tag_users.items():
            tag_user_csums[tag] = len(users)
                    
    #对item进行打分     
    for tag,wut in user_tags[user].items():
        for item,wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            if item not in recommend_items:
                if method == "SimpleTagBased":
                    #SimpleTagBased分数为所有的wut*wti之和（wut:用户对某标签的使用次数；wti:商品被打上相同标签的次数）
                    recommend_items[item] = wut * wti
                elif method == "NormTagBased":
                    #NormTagBased分数为所有的(wut/user_tag_csum)*(wti/tag_item_csums)之和
                    #(user_tag_csum:该用户打的所有标签次数，tag_item_csums:该标签被打上的总的次数) 
                    recommend_items[item] = (wut/user_tag_csum)*(wti/tag_item_csums[tag])
                elif method == "TagBased_TFIDF":
                    #TagBased_TFIDF分数为所有的(wut/log(1 + tag_user_csums))*(wti)之和（tag_user_csums:所有使用该标签的用户）
                    recommend_items[item] = (wut/np.log(1 + tag_user_csums[tag]))*wti
            else:
                if method == "SimpleTagBased":
                    recommend_items[item] += wut * wti
                elif method == "NormTagBased":
                    recommend_items[item] += (wut/user_tag_csum)*(wti/tag_item_csums[tag])
                elif method == "TagBased_TFIDF":
                    recommend_items[item] += (wut/np.log(1+tag_user_csums[tag]))*wti
    return sorted(recommend_items.items(),key = operator.itemgetter(1),reverse = True)[0:N]

In [8]:
#使用测试集计算精确率和召回率
#@njit(fastmath = True)
def precisionAndRecall(method,N):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        #获取topN推荐列表
        rank = recommend(method,user,N)
        for item,rui in rank:
            if item in items:
                hit = hit +1
        h_recall = h_recall + len(items)
        h_precision = h_precision + N
    print('一共命中{}个，一共推荐{}个，用户设置tag总数{}个'.format(hit,h_precision,h_recall))
    #返回精确率和召回率
    return (hit/(h_precision*1.0)),(hit/(h_recall*1.0))

In [9]:
#使用测试集对推荐结果进行评估
def testRecommend(method):
    print("{}推荐结果评估：".format(method))
    print("%3s %10s %10s"%("N","精确率","召回率"))
    for n in [5,10,20,40,60,80,100]:
        precision,recall = precisionAndRecall(method,n)
        print("%3d %10.3f%% %10.3f%%"%(n,precision * 100,recall * 100))

In [10]:
#数据加载
loadData()

开始数据加载...
数据集大小为437593
设置tag的人数1867
数据加载完成



In [11]:
#训练集测试集拆分，20%测试集
trainTestSplit(0.2)

训练集样本数 1860,测试集样本数 1793


In [12]:
initStat()

user_tags,tag_items,user_items,tag_users初始化完成。
user_tags大小 1860,tag_items大小 36884,user_items大小 1860,tag_users大小 36884


In [13]:
methods = ["SimpleTagBased","NormTagBased","TagBased_TFIDF"]
for method in methods:
    testRecommend(method)

SimpleTagBased推荐结果评估：
  N        精确率        召回率
一共命中74个，一共推荐8930个，用户设置tag总数20861个
  5      0.829%      0.355%
一共命中113个，一共推荐17860个，用户设置tag总数20861个
 10      0.633%      0.542%
一共命中183个，一共推荐35720个，用户设置tag总数20861个
 20      0.512%      0.877%
一共命中272个，一共推荐71440个，用户设置tag总数20861个
 40      0.381%      1.304%
一共命中341个，一共推荐107160个，用户设置tag总数20861个
 60      0.318%      1.635%
一共命中395个，一共推荐142880个，用户设置tag总数20861个
 80      0.276%      1.893%
一共命中443个，一共推荐178600个，用户设置tag总数20861个
100      0.248%      2.124%
NormTagBased推荐结果评估：
  N        精确率        召回率
一共命中64个，一共推荐8930个，用户设置tag总数20861个
  5      0.717%      0.307%
一共命中94个，一共推荐17860个，用户设置tag总数20861个
 10      0.526%      0.451%
一共命中147个，一共推荐35720个，用户设置tag总数20861个
 20      0.412%      0.705%
一共命中209个，一共推荐71440个，用户设置tag总数20861个
 40      0.293%      1.002%
一共命中262个，一共推荐107160个，用户设置tag总数20861个
 60      0.244%      1.256%
一共命中322个，一共推荐142880个，用户设置tag总数20861个
 80      0.225%      1.544%
一共命中382个，一共推荐178600个，用户设置tag总数20861个
100      0.214%      1.831%
TagBased_